# NLP: Predicting Upvotes Based on Headline
## Introduction
Hacker News is a community where users can submit articles, and other users can upvote those articles. The articles with the most upvotes make it to the front page, where they're more visible to the community.
## Goal
In this project, I'll be predicting the number of upvotes articles received, based on their headlines. Because upvotes are an indicator of popularity, I'll discover which types of articles tend to be the most popular.
## Data

The data set consists of submissions users made to Hacker News from 2006 to 2015. Developer Arnaud Drizard used the Hacker News API to scrape the data, which can be found in one of his [GitHub repositories](https://github.com/arnauddri/hn).

* `submission_time` - When the article was submitted
* `upvotes` - The number of upvotes the article received
* `url` - The base URL of the article
* `headline` - The article's headline

In [8]:
import pandas as pd
import numpy as np

submissions = pd.read_csv("submissions.csv")
submissions.columns = ["submission_time", "upvotes", "url", "headline"]
submissions = submissions.dropna()
submissions.head()

,submission_time,upvotes,url,headline
0,2015-01-05T16:20:01.000Z,1,medium.com,Safe Conferences Are Deliberately Designed
1,2012-03-27T15:23:57Z,1,nikefreerun2shoesuk.com,Nike Free 7.0 V2 Men's Shoes In Black / Gray /...
2,2012-10-29T22:52:24Z,3,neillcorlett.com,y Own HTTP Daemon - Simple daemon in Linux x86...
3,2011-08-25T07:21:44Z,1,meetlipsticklesbians.com,emory card xbox 360
4,2011-01-09T09:59:34Z,2,arstechnica.com,OWC launching SandForce-based SSDs for latest ...


In [9]:
submissions.shape

(363967, 4)

This dataset is pretty large, I'm going to shuffle the rows of the data frame and use a quarter of the data for this project.

In [12]:
np.random.seed(1)
shuffled_index = np.random.permutation(submissions.index)
submissions = submissions.reindex(shuffled_index)

#number of rows I want
print(int(len(submissions) / 2))

181983


In [13]:
submissions = submissions.iloc[:181983]

### Data Preparation
My goal is to train a linear regression algorithm that predicts the number of upvotes a headline would receive. To do this, I'll need to convert each headline to a numerical representation. I will be using the 'bag of words' model, which represents each piece of text as a numerical vector.

In [14]:
tokenized_headlines = []
for item in submissions['headline']:
    tokenized_headlines.append(item.split())

#preview the data  
print(tokenized_headlines[0:5])

[['Square', 'Exploring', '2014', 'IPO', 'With', 'Banks'], ['Newyork', 'Best', 'Seller', 'List'], ['Fake', 'cellphone', 'towers', 'intercepting', 'your', 'phone', 'calls'], ['Joey', 'Hess', 'leaves', 'Debian'], ['X', 'on', 'Wayland']]


Now that I have my tokens, I know they will need some processing to help with making predictions later on. I will need to get rid of punctuation, and make all words lowercase for consistency.

In [17]:
punctuation = [",", ":", ";", ".", "'", '"', "’", "?", "/", 
               "-", "+", "&", "(", ")"]
clean_tokenized = []

for item in tokenized_headlines:
    tokens = []
    for token in item:
        token = token.lower()
        for punc in punctuation:
            token = token.replace(punc, "")
        tokens.append(token)
    clean_tokenized.append(tokens)
    
clean_tokenized.head()

KeyboardInterrupt: 

In [19]:
clean_tokenized[:5]

[['square', 'exploring', '2014', 'ipo', 'with', 'banks'],
 ['newyork', 'best', 'seller', 'list'],
 ['fake', 'cellphone', 'towers', 'intercepting', 'your', 'phone', 'calls'],
 ['joey', 'hess', 'leaves', 'debian'],
 ['x', 'on', 'wayland']]

Now I will retrieve all unique words from each headline, create a matrix, and assign those words as column headers. After, I will populate the matrix with the number of token occurences.

In [16]:
unique_tokens = []
single_tokens = []
for tokens in clean_tokenized:
    for token in tokens:
        #checking if token isn't in single_tokens
        if token not in single_tokens:
            #if it isn't append the token to single_tokens
            single_tokens.append(token)
            #if token is in single_tokens and not in unique_tokens
        elif token in single_tokens and token not in unique_tokens:
            #add the token to unique_tokens
            unique_tokens.append(token)

counts = pd.DataFrame(0, index=np.arange(len(clean_tokenized)), columns=unique_tokens)
counts.head()

KeyboardInterrupt: 

In [21]:
import datetime
start = datetime.datetime.now()

unique_tokens = []
for tokens in clean_tokenized:
    for token in tokens:
        if token not in unique_tokens:
            unique_tokens.append(token)
            
end = datetime.datetime.now()
print(end - start)

0:03:24.218557


In [22]:
unique_tokens = {}
start = datetime.datetime.now()
for tokens in clean_tokenized:
    for token in tokens:
        if token not in unique_tokens:
            unique_tokens[token] = True

end = datetime.datetime.now()
print(end - start)

0:00:00.252376


In [25]:
columns = unique_tokens.keys()
print(columns)

dict_keys(['square', 'exploring', '2014', 'ipo', 'with', 'banks', 'newyork', 'best', 'seller', 'list', 'fake', 'cellphone', 'towers', 'intercepting', 'your', 'phone', 'calls', 'joey', 'hess', 'leaves', 'debian', 'x', 'on', 'wayland', 'drawing', 'app', 'for', 'artists', 'debuts', 'android', 'tablets', 'tech', 'talks', 'should', 'be', 'less', 'technical', 'if', 'tim', 'burton', 'did', 'pokemon', 'this', 'hoodie', 'is', 'so', 'insanely', 'popular', 'you', 'have', 'to', 'wait', 'months', 'get', 'it', 'show', 'hn', 'from', 'rss', 'social', 'networks', 'time', 'altering', 'mirror', 'gives', 'beer', 'goggles', 'trader', 'joes', 'down', 'aking', 'cool', 'ideas', 'happen', 'studying', 'our', 'users', 'and', 'software', 'immigrants', 'fresh', 'highquality', 'free', 'wordpress', 'themes', 'collection', 'silicon', 'valley', 'turning', 'lives', 'into', 'an', 'asset', 'class', 'different', 'cancer', 'causes', 'why', 'programming', 'puzzles', 'code', 'golf', 'page', 'se', 'still', 'in', 'beta', 'peak